# premier programme

* écrire un programme «bonjour monde»

```C
#include <cstdio>
__global__ void bonjour()
{
  printf("bonjour de la part de %d sur le GPU!\n", threadIdx.x);
}
int main()
{
  bonjour<<<1, 8>>>();
  cudaDeviceSynchronize();
  return 0;
}
```
* le compiler et le lancer sur le GPU

```bash
nvcc -arch=sm_35 egunon.cu  -Wno-deprecated-gpu-targets && ./a.out
bonjour de la part de 0 sur le GPU!
bonjour de la part de 1 sur le GPU!
bonjour de la part de 2 sur le GPU!
bonjour de la part de 3 sur le GPU!
bonjour de la part de 4 sur le GPU!
bonjour de la part de 5 sur le GPU!
bonjour de la part de 6 sur le GPU!
bonjour de la part de 7 sur le GPU!
```


# introspection du GPU
    
* copier l'archive 
```bash
# # cp /home/fuentes/deviceQuery.tgz .
```    
    
    
* decompresser l'archive du programme `deviceQuery` sur la frontale
```bash
tar xvzf deviceQuery.tgz  && cd deviceQuery && make
```

* executer deviceQuery par lots ou en mode interactif    
```bash
./deviceQuery
```

# Exemple de noyau élémentaire : homothétie

* coder le noyau homothetie

* coder une version CPU 

* comparer les 2 versions et verifier avec un assert que le calcul sur GPU est correct

* implanter une version retournée pour un pas de 2 du noyau

* comparer les perfs avec les fonctions de l'API

* comparer les perfs avec nvprof




# Array of Structs ou Struct of Arrays : c'est la question 😀

* on va coder nos particules de 2 façons
  * un tableau de structures
```C
struct AOS
{
  float x, y, z;
  float dx, dy, dz;
};
```
  * ou une structure de tableau
```C
struct SOA
{
  float * x, *y, *z;
  float * dx, *dy, *dz;
};
```
* l'algorithme va devoir faire $ x <- x + dx $, $y <- y + dy $, $z <- z + dz$ 

* coder une version utilisant AOS

* coder une version utilisant SOA

* s'assurer sur des petit cas qu'on calcule la meme chose.

* comparer les perfs


# Optimization 
## bande passante
- copier et décompresser `BP.tgz` et compiler le code (attention au module cuda)
```bash
# cp /home/fuentes/BP.tgz . && tar xvzf BP.tgz && cd bandePassante && make
```
- executer les 2 codes avec le profileur 
```bash
nvprof ./pratiqueFM 
nvprof ./pratique
```
- relever les temps de chaque noyau et retrouver les résultats de BP réelle données en cours pour le K40M

$$ \frac{\#R_{ker} + \#W_{ker}}{10^9t} $$
 - Attention le nombre d'écriture et de lecture est en octets!

- à comparer avec la bande passante max théorique
```bash
./theo
```

- décommenter l'option de compilation `--ptxas-options=-v` dans le makefile et recompiler
```bash
make 2>&1 | c++filt
```
- Que vous dit le compilo ?

## Transferts asynchrones
### Mémoire verrouillée
- copier et décompresser async.tgz
```bash
# cd ..
# cp /home/fuentes/async.tgz . && tar xvzf async.tgz && cd async
```
- compiler
```
make 
```
- profiler `./pinned.x` avec une trace
```
nvprof --print-gpu-trace ./pinned.x
```
- dans la sortie, pretez attention aux colonnes `SrcMemType`, `Troughut` et `Duration`

- Qu'en deduisez vous ?

- optionnel (pour les courageux(ses)) : 💪
    - essayer de changer l'enum `cudaHostAllocDefault` par `cudaHostAllocMapped`
    - supprimer les transferts de l'hôte vers GPU (les 2 `cudaMemcpy`)
    - écrire un noyau bidon (scale ou increment)
    - et verifier que la mémoire sur l'hôte est bien modifiée par le noyau

### transferts asynchrones
- lancer le programme `exAsync` et verifier que la version asynchrone est plus courte
```
./exAsync
```

- profiler le programme `exAsync.x`
```bash
nvprof --print-gpu-trace ./exAsync.x
```
- reconnaissez vous les differents appels du noyau ? notamment l'appel asynchrone en 4 morceaux

- changer le nombre de morceaux dans le sources et recompiler le programme, est plus rapide ou plus lent ?

#### note pour un affichage graphique
- Quand on dispose de `nvvp` en local (installer cuda sur sa machine). On peut extraire une trace 
```bash
nvprof --output-profile async.prof ./exAsync.x
```
- rapatrier la trace avec un scp et l'ouvrir avec nvvp
```
nvvp async.prof
```

## mémoire partagée : exemple de la transposition

- copier et decompresser les sources de la tranpostion
```bash
# cd ..
# cp /home/fuentes/transpose.tar . && tar xvf transpose.tar && cd tranposeSansSol
```

- compiler et faire tourner le programme
```
make
./transpose.x data/vinhamala.pgm
```
- charger le module feh et voir l'image transposée
```bash
module load visu/feh
feh --zoom 80 data/vinhamalaTrans.pgm
```
- écrire le noyau `transposeShared`, recompiler et l'executer
```bash
make
./transpose.x data/vinhamala.pgm 1
```
- verifier que le temps est inférieure à celui du noyau naif
- tester sans la synchronisation et voir que l'image produite est toute brouillée

### bonus (noyau sans conflits de bancs mémoire)
- copier le code du noyau `tranposeShared` dans `transposeNoConflicts` sans effacer la premier ligne avec `TILE_DIM+1`
- recompiler et utiliser le noyau
```bash
make
./transpose.x data/vinhamala.pgm 2
```
- verifier qu'il est encore plus rapide que `transposeShared`

- on peut aussi tester avec l'image `data/che.pgm` qui est encore plus grande

# calcul de π par réduction
- copier et décompresser  et compiler les sources de la reduction 
```bash
# cp /home/fuentes/reductions.tgz . && tar xvzf reductions.tgz && cd reductions && make
```
- tester le programme `monteCarlo.x` pour différentes valeurs de taille (premier argument)
- écrire un noyau `partialSumDivergent` qui applique le schéma divergent vu en cours. On pourra deverminer avec 4 blocs de 16 threads avec `cuda-gdb`
- écrire un noyau `partialSumDummy` où un unique thread calcule la somme
- en utilisant l'astuce des pointeurs vu au TP précedent (transposition), écrire un programme qui chronomètre les 3 noyaux et retrouver les résultats vu en cours

# Équation de la chaleur en 3D
- on veut résoudre l'équation de la chaleur en 3D
$$
\frac{\partial u}{\partial t} = \Delta u
$$
sur le cube $K = [0,1]^3$ avec 
 - une condition aux limites $u_{\partial K} = 1$ sur le bord 
 - une condition initiale $u_{\mathring{K}}(t=0) = 0$, $u_{\partial K}(t=0) = 1$

- on utilise le pochoir à 7 points suivant
![pochoir](pochoir7.svg)

- qui correspond à la discrétisation suivante 
$$
\frac{u_{i,j,k}^{n+1}-u_{i,j,k}^n}{\delta t} = 
\frac{u_{i+1,j,k}^n+u_{i-1,j,k}^n}{\delta x^2}+
\frac{u_{i,j+1,k}^n+u_{i,j+1,k}^n}{\delta y^2}+ 
\frac{u_{i,j,k+1}^n+u_{i,j,k+1}^n}{\delta z^2}\\
-(\frac{2}{\delta x^2}+\frac{2}{\delta y^2}+\frac{2}{\delta z^2})u_{i,j,k}^n
$$
- copier et décompresser les sources de la reduction
```bash
# cp /home/fuentes/chaleur.tgz . && tar xvzf chaleur.tgz && cd chaleur
```
- compiler et executer la version CPU
```bash
make && ./heatCPU.x
```
- écrire une version GPU du noyau et la tester avec le programme `heatTest.x`

- écrire une version GPU du programme principale avec sauvegarde de la solution dans des fichiers VTK
